In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from shapely.geometry import Point
import geopandas as gpd
import pandas as pd
import contextily as cx
from pygris import tracts, counties
import re
gpd.options.io_engine = "pyogrio"

In [ ]:
# Read in baltimore census tracts/counties
census_tracts = tracts(state = 'MD', cache = True, year = 2020).to_crs(32618)
census_tracts = census_tracts.rename(columns = {'GEOID':'GEOID.Tract'})
county_shapes = counties(state = 'MD', cache = True, year = 2020).to_crs(32618)
county_shapes = county_shapes[county_shapes['NAME'] == 'Baltimore']

# Clip the tracts for Baltimore counties, drop non polygons that interfere with overlay later
census_tracts = census_tracts.clip(county_shapes)
census_tracts = census_tracts[census_tracts.geom_type == 'Polygon']

In [11]:
# Import census population data and join to census tracts
from census import Census
from us import states
#variables reference page: https://api.census.gov/data/2020/acs/acs5/variables.html
#B09001_001E Total Pop. Under 18
#B09001_003E Pop. Age under 3
#B09001_004E Pop. Age 3-4
#B09001_005E Pop. Age 5
key = Census("da8e34e18cadd48c6ea83e5377334b54d338effa")
census_data = key.acs5.state_county_tract(fields = ('NAME', 'B09001_001E', 'B09001_003E', 'B09001_004E', 'B09001_005E'),
                                      state_fips = states.MD.fips,
                                      county_fips = '*',
                                      tract = "*",
                                      year = 2020)

census_data = pd.DataFrame(census_data)
# Create 5 and under column by adding together age 5, 3-4, and under 3
census_data['5 and Under'] = census_data[['B09001_003E', 'B09001_004E', 'B09001_005E']].sum(axis = 1)
# Create GEOID.Tract column to join to census spatial data
census_data['GEOID.Tract'] = census_data[['state', 'county', 'tract']].sum(axis = 1)
# Subset data to make output less cluttered
census_data = census_data.loc[:, ['B09001_001E', '5 and Under', 'GEOID.Tract']].rename(columns = {'B09001_001E': 'Total Population Under 18'})

# Join census population data and spatial data
tracts_pop_2020 = pd.merge(census_tracts, census_data, on = 'GEOID.Tract')
# Create a label column of population to overlay on each tract in maps
# tracts_pop_2020['label'] = tracts_pop_2020['5 and Under'].apply(lambda x: str(round(x)))
# Convert area to square miles from meters
tracts_pop_2020['area_sqmi'] = tracts_pop_2020.area/2589988
# Calculate population per square mile
tracts_pop_2020['5 and Under Per Sq. Mi'] = (tracts_pop_2020['5 and Under']/tracts_pop_2020['area_sqmi']).round(2)

In [8]:
# Read in csv with national road emission rates
all_vehicle_emission_rates = pd.read_csv('../national_hpms_data/national_hpms_emissions_all_vehicles.csv')
# Read in national 2018 hpms roadways and clip to our area of interest
hpms_roads = gpd.read_file('../national_hpms_data/hpms_all_states_geometry.gdb').to_crs(32618).clip(census_tracts)
# Merge roadways and emission rates
hpms_roads = pd.merge(hpms_roads, all_vehicle_emission_rates, on = 'FID_Link_Cnty_Intxn')

In [12]:
# Overlay returns roadways clipped to each census tract with identifying information
tract_emissions = gpd.overlay(hpms_roads, tracts_pop_2020)

# Get length of each roadway segment and multiply by emission rates
# Total emissions are then converted to tons per year
tract_emissions['clipped_length'] = tract_emissions.length
tract_emissions['PM10 tons/year (LDV, MDV, HDV combined)'] = (tract_emissions['clipped_length'] * tract_emissions['ER_PM10'] * 365 / 907185).round(4)
tract_emissions['PM2.5 tons/year (LDV, MDV, HDV combined)'] = (tract_emissions['clipped_length'] * tract_emissions['ER_PM25'] * 365 / 907185).round(4)
tract_emissions['NOx tons/year (LDV, MDV, HDV combined)'] = (tract_emissions['clipped_length'] * tract_emissions['ER_3_NOX'] * 365 / 907185).round(4)
tract_emissions['NO2 tons/year (LDV, MDV, HDV combined)'] = (tract_emissions['clipped_length'] * tract_emissions['ER_33_NO2'] * 365 / 907185).round(4)

# Aggregate road segment emissions by tract
tract_emissions = tract_emissions.groupby('GEOID.Tract').agg(
    {'clipped_length':'sum','PM10 tons/year (LDV, MDV, HDV combined)': 'sum', 'PM2.5 tons/year (LDV, MDV, HDV combined)':'sum',
    'NOx tons/year (LDV, MDV, HDV combined)':'sum', 'NO2 tons/year (LDV, MDV, HDV combined)':'sum'}).reset_index()

# Join population data to emissions data
tract_emissions = pd.merge(tracts_pop_2020, tract_emissions)

# Create columns for emissions per square mile and round to 2 places
tract_emissions[['PM10 tons/year (LDV, MDV, HDV combined) / SQMI',
                'PM2.5 tons/year (LDV, MDV, HDV combined) / SQMI',
                'NOx tons/year (LDV, MDV, HDV combined) / SQMI',
                'NO2 tons/year (LDV, MDV, HDV combined) / SQMI']] = tract_emissions.loc[:,
                ['PM10 tons/year (LDV, MDV, HDV combined)',
                'PM2.5 tons/year (LDV, MDV, HDV combined)',
                'NOx tons/year (LDV, MDV, HDV combined)',
                'NO2 tons/year (LDV, MDV, HDV combined)']].div(tract_emissions['area_sqmi'], axis = 0).round(2)

# Subset columns and write csv
# tract_emissions.loc[:, ['GEOID.Tract', 'NAMELSAD', 'Total Population Under 18', '5 and Under','area_sqmi', '5 and Under Per Sq. Mi',
# 'PM10 tons/year (LDV, MDV, HDV combined)', 'PM2.5 tons/year (LDV, MDV, HDV combined)', 'NOx tons/year (LDV, MDV, HDV combined)',
# 'NO2 tons/year (LDV, MDV, HDV combined)', 'PM10 tons/year (LDV, MDV, HDV combined) / SQMI', 'PM2.5 tons/year (LDV, MDV, HDV combined) / SQMI', 
# 'NOx tons/year (LDV, MDV, HDV combined) / SQMI', 'NO2 tons/year (LDV, MDV, HDV combined) / SQMI']].to_csv('./outputs/vehicle_emissions_by_tract.csv', index = False)

In [ ]:
# Plot each variable by tract
from matplotlib_scalebar.scalebar import ScaleBar
import matplotlib.patches as mpatches

for column in tract_emissions.loc[:, ['5 and Under Per Sq. Mi', 'PM10 tons/year (LDV, MDV, HDV combined) / SQMI', 'PM2.5 tons/year (LDV, MDV, HDV combined) / SQMI', 
'NOx tons/year (LDV, MDV, HDV combined) / SQMI', 'NO2 tons/year (LDV, MDV, HDV combined) / SQMI']]:

        fig, ax = plt.subplots(figsize = (10,8), layout = 'constrained')

        # Add 2 mile scalebar by creating 2 points in a crs using feet
        points = gpd.GeoSeries([Point(3291352.870, 654597.160), Point(3606764.130, 674249.108)], crs=3452)
        distance_feet = points[0].distance(points[1])
        ax.add_artist(ScaleBar(distance_feet, dimension="imperial-length", units="ft", fixed_units = "mi", fixed_value = 2))

        # Plot census tracts with emissions
        tract_emissions.to_crs(4326).plot(column, cmap = 'inferno_r', legend = True, ax = ax, aspect = 1)
        # Plot census tract outlines
        tract_emissions.to_crs(4326).plot(column, facecolor = 'none', edgecolor = 'black', linewidth = 0.5, legend = False, ax = ax, aspect = 1)
        # Add OpenStreetMap basemap
        cx.add_basemap(ax = ax, zoom = 12, source=cx.providers.OpenStreetMap.Mapnik,
                interpolation = 'sinc', crs = 4326)
        
        # Set title using the column name
        ax.set_title(str(column) + ' by Census Tract', size = 14)

        # Remove ticks around the figure
        plt.tick_params(left = False, right = False , labelleft = False , 
                labelbottom = False, bottom = False)
        
        # Move and reduce the size of the basemap credits
        txt = ax.texts[-1]
        txt.set_fontsize(7)
        txt.set_position([0.55,0.01])
        txt.set_ha('left')
        txt.set_va('bottom')

        # Save each figure under outputs using the pollutant name
        plt.savefig('./outputs/' + re.search(r'[^ ]*', column).group() + '_vehicle_emissions_tract.png', dpi=300)
        plt.close()